In [ ]:
#Mendownload dataset 
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip  \
  -O /tmp/rockpaperscissors.zip

In [ ]:
#Mengimport modul yang akan digunakan
import tensorflow as tf
import os
import zipfile
import keras.preprocessing
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Mengekstrak dataset
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
# Melakukan Augmentasi gambar
Base_dir ='/tmp/rockpaperscissors/rps-cv-images'
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    validation_split = 0.4,
                    fill_mode = 'nearest')

In [ ]:
#Mempersiapkan data latih
train_generator = train_datagen.flow_from_directory(
        Base_dir,  
        target_size=(150, 150), 
        shuffle= None,
        class_mode='categorical',
        subset= 'training')

validation_generator = train_datagen.flow_from_directory(
        Base_dir, 
        target_size=(150, 150),
        shuffle= None, 
        class_mode='categorical',
        subset='validation')

In [ ]:
#Membangun arsitektur model dengan CNN.
model = tf.keras.models.Sequential([    
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
#Melakukan loss function dan Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(train_generator, 
          steps_per_epoch= 4,
          validation_steps=4, 
          validation_data = validation_generator, 
          epochs=150,
          verbose=2)

In [ ]:
#Menguji model dengan data baru
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
 
uploaded = files.upload()
 
for fn in uploaded.keys():
    path = fn
    img = image.load_img(path, target_size=(150,150))
    imgplot = plt.imshow(img)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
 
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
  
    print(fn)
    if classes [0][0]== 1:
        print('Paper')
    elif classes [0][1]==1:
        print('Rock')
    elif classes [0][2]==1:
        print('Scissors')